In [93]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

In [94]:
#data quick summary function

def df_quick_info(df):
    report = pd.DataFrame(columns=['Column', 'Data Type', 'Unique Count', 'Unique Sample', 'Missing Values', 'Missing Percentage', 'Unique Percentage'])
    for column in df.columns:
        data_type = df[column].dtype
        unique_count = df[column].nunique()
        unique_sample = df[column].unique()[:5]
        missing_values = df[column].isnull().sum()
        missing_percentage = (missing_values / len(df)) * 100
        unique_percentage = (df[column].nunique() / len(df)) * 100
        report = pd.concat([report, pd.DataFrame({'Column': [column],
                                                      'Data Type': [data_type],
                                                      'Unique Count': [unique_count],
                                                      'Unique Sample': [unique_sample],
                                                      'Missing Values': [missing_values],
                                                      'Missing Percentage': [missing_percentage.round(4)],
                                                      'Unique Percentage': [unique_percentage]})],
                            ignore_index=True)
    return report

In [95]:
df = pd.read_excel('datasets/Online Retail Data Set.xlsx', sheet_name='Online Retail')

In [96]:
display(df.head(5))
print(df.shape)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


(541909, 8)


In [97]:
df_quick_info(df)

,Column,Data Type,Unique Count,Unique Sample,Missing Values,Missing Percentage,Unique Percentage
0,InvoiceNo,object,25900,"[536365, 536366, 536367, 536368, 536369]",0,0.0000,4.779400
1,StockCode,object,4070,"[85123A, 71053, 84406B, 84029G, 84029E]",0,0.0000,0.751049
2,Description,object,4223,"[WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET...",1454,0.2683,0.779282
3,Quantity,int64,722,"[6, 8, 2, 32, 3]",0,0.0000,0.133233
4,InvoiceDate,datetime64[ns],23260,"[2010-12-01T08:26:00.000000000, 2010-12-01T08:...",0,0.0000,4.292234
5,UnitPrice,float64,1630,"[2.55, 3.39, 2.75, 7.65, 4.25]",0,0.0000,0.300789
6,CustomerID,float64,4372,"[17850.0, 13047.0, 12583.0, 13748.0, 15100.0]",135080,24.9267,0.806778
7,Country,object,38,"[United Kingdom, France, Australia, Netherland...",0,0.0000,0.007012


In [98]:
filtered_df = df[
    ~(
        df['Description'].isna() |
        df['Description'].apply(lambda x: 'wrong' in str(x).lower() or 'damage' in str(x).lower() or 'crushed' in str(x).lower() or '?' in str(x).lower()))
]

items = pd.DataFrame(filtered_df.groupby('StockCode')['Description'].apply(lambda x: x.astype('str').max())).reset_index()

items['Description'] = items.Description.replace('nan', pd.NA)

df = df.merge(items, on='StockCode', how = 'left')



In [99]:
for i in set(df.query('Description_x.isna()').Description_y):
    print(i)

RETRO TIN ASHTRAY,REVOLUTIONARY
WHITE DOVE HONEYCOMB PAPER GARLAND
GLITTER HEART GARLAND WITH BELLS 
YOU'RE CONFUSING ME METAL SIGN 
MINI HIGHLIGHTER PENS
DAIRY MAID  PUDDING BOWL
ORANGE TV TRAY TABLE 
SET OF 6 SPICE TINS PANTRY DESIGN
SMALL JEWELLERY STAND
METAL TUBE CHIME ON BAMBOO
SILVER BOOK MARK WITH BEADS
CHILDS GARDEN BRUSH PINK
NEWSPAPER STAND
POTTING SHED SOW 'N' GROW SET
FROG SOCK PUPPET
SCENTED CANDLE IN DIGITALIS TIN
TEATIME GEL PENS ASST
PASTEL BLUE PHOTO ALBUM 
UNION FLAG WINDSOCK
ROBIN CHRISTMAS CARD
FOLDING CAMPING SCISSOR W/KNIF & S
VINTAGE KITCHEN PRINT FRUITS
BOX/12 CHICK & EGG IN BASKET
SET/3 POLKADOT STACKING TINS
POP ART PUSH DOWN RUBBER 
TEATIME FUNKY FLOWER BACKPACK FOR 2
VINTAGE BILLBOARD TEA MUG
ICON MUG REVOLUTIONARY
DOLPHIN WINDMILL
PURPLE ANEMONE ARTIFICIAL FLOWER
AIRLINE LOUNGE,METAL SIGN
CHRISTMAS GARLAND STARS,TREES
SET OF 5 PANCAKE DAY MAGNETS
15 PINK FLUFFY CHICKS IN BOX
SET/20 STRAWBERRY PAPER NAPKINS 
HEART OF WICKER SMALL
HANGING HEART BASKET
HALL C

In [100]:
def clean_descr(row):
    orig = row['Description_x']
    clean = row['Description_y']
    if pd.isna(orig):
        return clean
    else:
        return orig
df['Description'] = df.apply(clean_descr, axis=1)

df = df.drop(columns=['Description_x','Description_y'])

In [105]:
customers = pd.DataFrame(df[~df['CustomerID'].isna()].groupby('InvoiceNo')['CustomerID'].max()).reset_index()

customers['CustomerID'] = customers.CustomerID.replace('nan', pd.NA)

df = df.merge(customers, on='InvoiceNo', how = 'left')

In [108]:
def clean_cust(row):
    orig = row['CustomerID_x']
    clean = row['CustomerID_y']
    if pd.isna(orig):
        return clean
    else:
        return orig
df['CustomerID'] = df.apply(clean_cust, axis=1)

df = df.drop(columns=['CustomerID_x','CustomerID_y'])

In [111]:
df_quick_info(df)

,Column,Data Type,Unique Count,Unique Sample,Missing Values,Missing Percentage,Unique Percentage
0,InvoiceNo,object,25900,"[536365, 536366, 536367, 536368, 536369]",0,0.0000,4.779400
1,StockCode,object,4070,"[85123A, 71053, 84406B, 84029G, 84029E]",0,0.0000,0.751049
2,Quantity,int64,722,"[6, 8, 2, 32, 3]",0,0.0000,0.133233
3,InvoiceDate,datetime64[ns],23260,"[2010-12-01T08:26:00.000000000, 2010-12-01T08:...",0,0.0000,4.292234
4,UnitPrice,float64,1630,"[2.55, 3.39, 2.75, 7.65, 4.25]",0,0.0000,0.300789
5,Country,object,38,"[United Kingdom, France, Australia, Netherland...",0,0.0000,0.007012
6,Description,object,4223,"[WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET...",112,0.0207,0.779282
7,CustomerID,float64,4372,"[17850.0, 13047.0, 12583.0, 13748.0, 15100.0]",135080,24.9267,0.806778


In [112]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [114]:
numeric_cols  = [var for var in df.select_dtypes(include='number')]

In [115]:
numeric_cols

['Quantity', 'UnitPrice', 'CustomerID']